In [1]:
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

### 1. 실시간 뉴스 요약 챗봇

- 사용자가 입력한 키워드(예: "삼성전자", "AI", "테슬라")에 대한 최신 뉴스를 검색하고, 그 결과를 한글로 요약해서 보여주는 챗봇 제작

# 검색api로 뉴스 불러오기 -> 프롬프트 설정(요약,출력 방식)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

search = GoogleSerperAPIWrapper(type= "news",tbs="qdr:h")

llm = ChatOpenAI(model="gpt-4.1-nano")

prompt = ChatPromptTemplate.from_template(
    "다음 검색 결과를 한글로 300자로 요약해줘 그 근거가 되는 검색결과도 같이 반환해:\n{search_results}"
)

chain = {'search_results': RunnableLambda(search.run)} | prompt | llm | StrOutputParser()


In [28]:
user_input = "삼성전자"
result = chain.invoke(user_input)
print(search.results(user_input))
print(result)

{'searchParameters': {'q': '삼성전자', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'tbs': 'qdr:h', 'engine': 'google'}, 'organic': [{'title': '삼성전자 (005930) - 인베스팅', 'link': 'https://kr.investing.com/equities/samsung-electronics-co-ltd', 'snippet': '삼성전자의 주가, 005930 주식, 차트, 기술적 분석, 실적 자료 등 69100로 거래를 마감한 삼성전자 시세에 대한 자세한 정보를 확인해 보세요. 오늘의 시세뿐만 아니라 ...', 'date': '2 minutes ago', 'sitelinks': [{'title': '차트', 'link': 'https://kr.investing.com/equities/samsung-electronics-co-ltd-chart'}, {'title': '배당', 'link': 'https://kr.investing.com/equities/samsung-electronics-co-ltd-dividends'}, {'title': '재무 정보 요약', 'link': 'https://kr.investing.com/equities/samsung-electronics-co-ltd-financial-summary'}, {'title': '기술적 분석', 'link': 'https://kr.investing.com/equities/samsung-electronics-co-ltd-technical'}], 'position': 1}, {'title': '직무추천서비스 - 삼성전자 DS부문 채용', 'link': 'https://www.samsung-dsrecruit.com/recruits/job_description/index.php', 'snippet': '삼성전자 DS부문의 직무추천서비스 페이지로, 지원자의 전공과 흥미도를 분석하여 적합한

### 2. 오늘의 환율/주가/날씨 안내 서비스

- "오늘 원/달러 환율 알려줘", "오늘 삼성전자 주가 알려줘", "서울 날씨 알려줘" 등 실시간 데이터를 검색해서 답변하는 서비스 구축

# 환율 / 주가 / 날씨 셋 중 하나로 분류 하는 llm(꼭 필요?) -> 각 분야마다 답변하는 llm & 프롬프트 설정

In [89]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

search = GoogleSerperAPIWrapper(type="search",tbs="qdr:3h")

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template("다음 검색 결과를 이용해서 사용자의 질문에 대해 한줄로 답변해.\
                                          검색 결과에 명시적으로 사용자의 질문과 관련된 언급이 없다면 해당 정보를 사용하지마:\
                                            검색결과 : {search_results}, 사용자 질문: {user_input}")

user_input = '오늘 엔화 환율 알려줘'

# 검색 결과를 'user_input' 기반으로 생성
search_lambda = RunnableLambda(lambda x: search.run(x["user_input"]))

# 체인 구성
chain = {"search_results": search_lambda, "user_input": lambda x: x["user_input"]} | prompt | llm | StrOutputParser()

result = chain.invoke({"user_input": user_input})

print(search.results(user_input))
print(result)

{'searchParameters': {'q': '오늘 엔화 환율 알려줘', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'tbs': 'qdr:3h', 'engine': 'google'}, 'organic': [{'title': 'JPY KRW 오늘 | 엔 원 환율 - Investing.com - 인베스팅', 'link': 'https://kr.investing.com/currencies/jpy-krw', 'snippet': 'JPY/KRW - 일본 엔 원 ; 전일 종가. 9.3968 ; 매수: 9.4156 ; 금일 변동: 9.3897 ; 금일 시가. 9.3968 ; 매도: 9.4202.', 'sitelinks': [{'title': '환율 변환기', 'link': 'https://kr.investing.com/currencies/jpy-krw-converter'}, {'title': '차트', 'link': 'https://kr.investing.com/currencies/jpy-krw-chart'}, {'title': '포럼', 'link': 'https://kr.investing.com/currencies/jpy-krw-commentary'}, {'title': '기술 분석', 'link': 'https://kr.investing.com/currencies/jpy-krw-technical'}], 'position': 1}, {'title': '일본 엔 → 대한민국 원 환율. JPY/KRW 환전 - Wise', 'link': 'https://wise.com/kr/currency-converter/jpy-to-krw-rate', 'snippet': 'Wise 환율계산기로 JPY → KRW 변환을 하세요. 이전 환율 차트 또는 실시간 일본 엔 / 대한민국 원 환율을 분석하고 무료 환율 알림을 이메일로 직접 받아볼 수 있습니다.', 'sitelinks': [{'title': '10000 JPY', 'link': 

### 3. 인물/연예/스포츠 최신 이슈 Q&A

- "2024 파리 올림픽에서 한국이 딴 메달 수는?", "최근 노벨상 수상자는?" 등 최신 이슈를 검색해서 답변

# 2와 동일

### 4. 신제품/기술 동향 리포트 생성기

- "2024년 애플 신제품 소식 요약", "최근 출시된 전기차 비교" 등 최신 기술/제품 동향을 검색하고 요약

# 2와 동일

### 5. 실시간 이미지/장소/뉴스 검색 챗봇

- "강아지 사진 보여줘", "서울 강남 맛집 추천해줘", "테슬라 관련 최신 뉴스 알려줘" 등 다양한 유형의 검색 결과를 받아서 보여주기

# 사용자의 질문에 따라서 type을 결정 -> 검색

In [58]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser
import requests


# OPEN AI API 키 설정
load_dotenv()


MY_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=MY_API_KEY)

def classify_type_gpt(user_query):
        user_prompt = f'''
        당신은 사용자의 질문을 4종류로 분류하는 판별가 입니다. 
        분류는 다음과 같습니다.
        - search, news, images, places
        1개의 질문은 1개의 종류만 가능합니다.
        사용자의 질문 : {user_query}
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요.
        'type' : 분류
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        return json.loads(response.choices[0].message.content)

#######################################
user_input = ' 뉴스 알려줘'
#######################################

classified_type = classify_type_gpt(user_input)['type']
print(classified_type)


search = GoogleSerperAPIWrapper(type=classified_type, tbs="qdr:h")

llm = ChatOpenAI(model="gpt-4o")

if classified_type == "images":
    results = search.results(user_input)
    image_url = results['images'][0]['imageUrl']
    r = requests.get(image_url,headers = { 'user-agent': 'Mozilla 5.0'})
    with open(f'{user_input}.jpg','wb') as f:
        f.write(r.content)
else:
    if classified_type == "news":
        type_prompt = "다음 검색 결과를 바탕으로 뉴스를 요약해줘 \n 검색결과:{search_results}"
    elif classified_type == "places":
        type_prompt = "다음 검색 결과를 바탕으로 장소를 추천해줘\n 검색결과:{search_results}"
    elif classified_type == "search":
        type_prompt = "다음 검색 결과를 요약해줘\n 검색결과:{search_results}"

    prompt = ChatPromptTemplate.from_template(type_prompt)
    chain = {'search_results': RunnableLambda(search.run)} | prompt | llm | StrOutputParser()
    result = chain.invoke(user_input)
    print(result)
    print(search.results(user_input))


news
광주시 광산구가 올해도 '어르신의 안부를 묻는 우유배달' 사업을 지속합니다. 이 사업은 작년에 시작되었으며, 3개 권역의 84가구를 대상으로 진행되고 있습니다. 이를 통해 어르신들의 안부를 확인하고 이들의 복지를 지원하고자 하는 목적을 가지고 있습니다.
{'searchParameters': {'q': ' 뉴스 알려줘', 'gl': 'us', 'hl': 'en', 'type': 'news', 'num': 10, 'page': 1, 'tbs': 'qdr:h', 'engine': 'google'}, 'news': [{'title': '광주시 광산구, 우유배달로 홀몸 어르신 안부 챙긴다~고독사 예방 앞장', 'link': 'https://www.wikitree.co.kr/articles/1071260', 'snippet': "[위키트리 광주전남취재본부 노해섭 기자]광주시 광산구(구청장 박병규)가 올해도 '어르신의 안부를 묻는 우유배달' 사업을 이어간다.지난해에 이어 3개 권역 84가구...", 'date': '49 minutes ago', 'source': '위키트리', 'position': 1}], 'credits': 1}


### 6. 실시간 이슈 기반 토론/퀴즈 챗봇

- "오늘의 주요 국제 뉴스로 퀴즈 내줘", "최근 이슈로 토론 주제 추천해줘" 등 실시간 이슈를 활용한 챗봇

# 실시간 이슈로 퀴즈,토론 주제 만들기 -> 검색된 이슈 기반으로 채점

In [92]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import JsonOutputParser
import requests


# OPEN AI API 키 설정
load_dotenv()


MY_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=MY_API_KEY)

def classify_type_gpt(user_query):
        user_prompt = f'''
        당신은 사용자의 질문을 정제,분류하는 판별가 입니다. 
        분류는 다음과 같습니다.
        type : quiz, discussion
        1개의 질문은 1개의 종류만 가능합니다.
        사용자의 질문을 검색 엔진에 넣기 적합한 형태로 가공하세요. 가공된 단어는 반드시 사용자의 질문 중 일부여야합니다. '퀴즈','토론' 단어는 제외하세요

        사용자의 질문 : {user_query}
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요.
        'type' : 분류
        'revised_keyword' : 가공된 단어
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        return json.loads(response.choices[0].message.content)

#######################################
user_input = '최근 이슈로 퀴즈 추천해줘'
user_input_revised = classify_type_gpt(user_input)['revised_keyword']
#######################################
print(user_input_revised)

classified_type = classify_type_gpt(user_input)['type']
print(classified_type)

search = GoogleSerperAPIWrapper(type='news', tbs="qdr:d")

llm = ChatOpenAI(model="gpt-4o")

if classified_type == "quiz":
    type_prompt = "다음 검색 결과로 퀴즈 1개를 4지선다형으로 만들어줘.\
                    해당 퀴즈에 대한 정답을 생성해.\
                    검색 결과가 없으면 다른 키워드를 요청해. \
                    검색 결과 외의 임의의 퀴즈는 반환하지마 \
                    검색결과:{search_results}\
                    출력은 반드시 json형태로 마크다운 없이 반환해.\
                    'quiz' : 퀴즈\
                    'choices': 보기\
                    'correct_answer' : 정답 숫자"
                    
elif classified_type == "discussion":
    type_prompt = "다음 검색 결과로 토론 주제 1개를 추천해줘.\
                    검색 결과가 없으면 다른 키워드를 요청해. \
                    검색 결과 외의 임의의 퀴즈는 반환하지마 \
                    검색결과:{search_results}\
                    출력은 반드시 json형태로 마크다운 없이 반환해.\
                    'discussion' : 토론 주제"
                    
prompt = ChatPromptTemplate.from_template(type_prompt)
chain = {'search_results': RunnableLambda(search.run)} | prompt | llm | JsonOutputParser()
result = chain.invoke(user_input_revised)
print(result)
print(search.results(user_input_revised))

def grading_discussion_gpt(discussion):
    user_argue = input()
    user_prompt = f'''
    당신은 토론 참여자가 토론 주제에 맞게 논리적으로 주장하였는지 검증하는 판별가 입니다. 
    토론 참여자의 점수를 0~100점으로 점수를 매기고 그 이유를 한줄로 서술하세요.
    토론 주제 : {discussion}
    토론 참여자의 주장 : {user_argue}
    
    출력은 반드시 json형태로 마크다운 없이 반환하세요.
    'score' : 점수
    'reason' : 이유
    '''

    response = client.chat.completions.create(model = "gpt-4o",
                                        messages = [
                                            {'role': 'user', 'content': user_prompt},
                                            ],
                                        max_tokens=150,
                                        temperature=0
                                        )
    return json.loads(response.choices[0].message.content)
if classified_type == "quiz":
    quiz_input = input()
    if int(result['correct_answer']) == int(quiz_input):
        print('정답입니다')
    else:
        print('오답입니다')

elif classified_type == "discussion":
    grading_result = grading_discussion_gpt(result['discussion'])
    print('점수:',grading_result['score'])
    print('이유:',grading_result['reason'])

최근 이슈
quiz
{'quiz': '여성 듀오 애즈원의 이민(본명 이민영)의 사망 소식은 어느 소속사를 통해 발표되었는가?', 'choices': ['티엔엔터테인먼트', '브랜뉴뮤직', 'SM엔터테인먼트', 'YG엔터테인먼트'], 'correct_answer': 2}
{'searchParameters': {'q': '최근 이슈', 'gl': 'us', 'hl': 'en', 'type': 'news', 'num': 10, 'page': 1, 'tbs': 'qdr:d', 'engine': 'google'}, 'news': [{'title': "'자택서 사망' 故 이민 누구?..최근 박보검과 무대한 애즈원 멤버 [스타이슈]", 'link': 'https://www.starnewskorea.com/star/2025/08/06/2025080617031881323', 'snippet': '여성 듀오 애즈원의 이민(본명 이민영)이 갑작스럽게 세상을 떠났다. 6일 소속사 브랜뉴뮤직에 따르면 이민은 지난 5일 오후 자택에서 숨진 채 발견됐다.', 'date': '17 minutes ago', 'source': '스타뉴스', 'position': 1}, {'title': "[이슈] 하루 만에 SNS 팔로워 10만…식지 않는 '모솔연애'", 'link': 'https://news.nate.com/view/20250806n30768', 'snippet': "한눈에 보는 오늘 : 연예가 화제 - 뉴스 : '모태솔로지만 연애는 하고 싶어' 모태솔로들이 글로벌 셀럽으로 거듭났다. 최근 막을 내린 넷플릭스 예능 '모태솔로지만...", 'date': '12 minutes ago', 'source': '네이트', 'position': 2}, {'title': '13일 ‘AI 시대 음원 저작권 보호 법적 대응과 과제’', 'link': 'https://www.munhaknews.com/news/articleView.html?idxno=81379', 'snippet': '[문학뉴스=남미리 기자